Number of labels: 31

In [1]:
! pip install -U accelerate
! pip install -U transformers

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)
#device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu')
#print(device)

Fri Dec  1 13:02:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    24W / 300W |      0MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!pip install -qq wandb --upgrade

In [4]:
import wandb
wandb.login()


%env WANDB_PROJECT=vit_snacks_sweeps
%env WANDB_LOG_MODEL=true

wandb: Currently logged in as: kheirkhahzade (deeplearning_kth). Use `wandb login --relogin` to force relogin


env: WANDB_PROJECT=vit_snacks_sweeps
env: WANDB_LOG_MODEL=true


In [5]:
!pip install --upgrade huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

packages we need to use HuggingFace API

In [6]:
!pip install transformers datasets tokenizers seqeval -q

In [7]:
import numpy as np
import datasets
from transformers import BertTokenizerFast                        # We need bert tokenizer
from transformers import DataCollatorForTokenClassification       # DataCollator for some tasks like data augmentation in NLP task (token classification)
from transformers import AutoModelForTokenClassification ,TFAutoModelForTokenClassification         # For loading model in the memory.

### Loading Dataset

In [8]:
en_dataset = datasets.load_dataset('MaryDatascientist/en_dataset')

#label_list = [i for i in range(31)]
#print(label_list)
label_list = [
    "O",
    "B-PER",
    "I-PER",
    "B-ORG",
    "I-ORG",
    "B-LOC",
    "I-LOC",
    "B-ANIM",
    "I-ANIM",
    "B-BIO",
    "I-BIO",
    "B-CEL",
    "I-CEL",
    "B-DIS",
    "I-DIS",
    "B-EVE",
    "I-EVE",
    "B-FOOD",
    "I-FOOD",
    "B-INST",
    "I-INST",
    "B-MEDIA",
    "I-MEDIA",
    "B-MYTH",
    "I-MYTH",
    "B-PLANT",
    "I-PLANT",
    "B-TIME",
    "I-TIME",
    "B-VEHI",
    "I-VEHI"]


In [9]:
id2label = {
    str(i): label for i,label in enumerate(label_list)
}
label2id = {
    label: str(i) for i, label in enumerate(label_list)
}

In [10]:
en_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 262560
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 32820
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 32908
    })
})

Initialize your tokenizer

In [11]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased') #or bert-base-uncased

Data Engineering for CLS, and SEP character to not be considered in training phase:

In [12]:
def resolving_cls_sep_labels(examples, label_all_tokens =True):
    tokenized_inputs = tokenizer(examples['tokens'],padding='max_length', max_length=64,
                                      truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples['ner_tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
               # set -100 as the label for these special tokens
               label_ids.append(-100)
            elif word_idx != previous_word_idx:
               label_ids.append(label[word_idx])
            else:
              label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs['labels'] = labels
    return tokenized_inputs

In [13]:
tokenized_en_datasets = en_dataset.map(resolving_cls_sep_labels, batched=True)

Map:   0%|          | 0/32820 [00:00<?, ? examples/s]

###Initialize your model

In [14]:
model = AutoModelForTokenClassification.from_pretrained('bert-base-uncased', num_labels=31)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Hyper-parameters

In [15]:
# method
sweep_config = {
    'method': 'random'
}


# hyperparameters
parameters_dict = {
    'epochs': {
        'value': 1
        },
    'batch_size': {
        'values': [ 32, 64]
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 2e-5,
        'max': 1e-3
    },
    'weight_decay': {
        'values': [0,0.01, 0.1]
    },
}


sweep_config['parameters'] = parameters_dict

In [16]:
sweep_id = wandb.sweep(sweep_config, project='modelA_1_12_2023_sweeps')

Create sweep with ID: 9v066ic0
Sweep URL: https://wandb.ai/deeplearning_kth/modelA_1_12_2023_sweeps/sweeps/9v066ic0


### Setting training arguments

In [17]:
#!pip install transformers[torch]#pip install accelerate -U#

In [18]:
data_collator = DataCollatorForTokenClassification(tokenizer = tokenizer)

In [19]:
metric = datasets.load_metric('seqeval')

<ipython-input-19-8729a7e2f3c1>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('seqeval')


In [20]:
def compute_metrics(eval_preds):

  pred_logits, labels = eval_preds
  pred_logits = np.argmax(pred_logits, axis = 2)

  predictions = [
      [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l !=-100]
      for prediction, label in zip(pred_logits, labels)
  ]
  true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l!=-100]
      for prediction, label in zip(pred_logits, labels)
  ]
  results =metric.compute(predictions=predictions, references=true_labels)
  return {
      'precision': results['overall_precision'],
      'recall': results['overall_recall'],
      'f1': results['overall_f1'],
      'accuracy': results['overall_accuracy'],
  }

### Trainer and Wandb

In [21]:
from transformers import TrainingArguments, Trainer

def train(config=None):
  with wandb.init(config=config):
    # Set sweep configuration
    config = wandb.config

    args = TrainingArguments(
    output_dir = 'modelA_1_12_2023',
    report_to='wandb',  # Turn on Weights & Biases logging

    #evaluation_strategy= 'epoch',
    evaluation_strategy="steps",

    learning_rate=config.learning_rate,
    per_device_train_batch_size=config.batch_size,
    per_device_eval_batch_size=16,

    logging_first_step=True,
    logging_steps=50,
    eval_steps=500,


    num_train_epochs= config.epochs,
    weight_decay=config.weight_decay,
    remove_unused_columns=True,
    load_best_model_at_end=True

    )
    trainer = Trainer(
        model,
        args,

        data_collator = data_collator,

        train_dataset = tokenized_en_datasets['train'],
        eval_dataset = tokenized_en_datasets['validation'],
        tokenizer = tokenizer,

        compute_metrics=compute_metrics
    )
    # Start training
    trainer.train()

    # Push to hub
    trainer.push_to_hub('MaryDatascientist/modelA_1_12_2023')

### Start hyperparameter tuning

In [22]:
wandb.agent(sweep_id, train, count=2)

wandb: Agent Starting Run: vlffnez5 with config:
wandb: 	batch_size: 64
wandb: 	epochs: 1
wandb: 	learning_rate: 0.00019616920113351048
wandb: 	weight_decay: 0


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.097600,0.105478,0.804727,0.802545,0.803634,0.964063
1000,0.084900,0.097433,0.818551,0.847901,0.832968,0.966703
1500,0.078800,0.086530,0.862255,0.827353,0.844443,0.969522
2000,0.067800,0.083117,0.886037,0.809231,0.845894,0.970483
2500,0.055500,0.079000,0.840133,0.889927,0.864313,0.971756
3000,0.046800,0.074507,0.872685,0.876943,0.874808,0.973936
3500,0.041700,0.070357,0.868385,0.894187,0.881097,0.975067
4000,0.035900,0.068873,0.878376,0.891528,0.884903,0.975977


training_args.bin:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

...

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.023600,0.080360,0.871890,0.885642,0.878712,0.974734
1000,0.023500,0.083978,0.846141,0.905597,0.874860,0.973461
1500,0.019300,0.086929,0.857206,0.898962,0.877588,0.974487
2000,0.017400,0.085275,0.868940,0.891192,0.879926,0.974860
2500,0.013600,0.089928,0.874994,0.892251,0.883538,0.975375
3000,0.011900,0.095428,0.868476,0.896432,0.882233,0.975147
3500,0.012600,0.091905,0.871104,0.893231,0.882029,0.975361
4000,0.019700,0.087536,0.876094,0.891631,0.883794,0.975618


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.54k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

eval/accuracy,▅▁▄▆▇▆▇█
eval/f1,▄▁▃▅█▇▇█
eval/loss,▁▃▄▃▅█▆▄
eval/precision,▇▁▄▆█▆▇█
eval/recall,▁█▆▃▃▅▄▃
eval/runtime,█▁▆▆▂▆▃▁
eval/samples_per_second,▁█▃▃▇▃▆█
eval/steps_per_second,▁█▃▃▇▃▆█
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
